In [5]:
from pathlib import Path

import nest_asyncio
import tqdm

nest_asyncio.apply()

import time

import arrow
import duckdb
import pandas as pd
import polars as pl

from measurement.lake.measurement_lake_controller import MeasurementLakeController
from measurement.util.logger import get_logger

log = get_logger(__name__)

In [2]:
do_download = True
lake = MeasurementLakeController.create(
    Path("/mnt/apple/cisco-umbrella-2019-2025"), output=Path("/mnt/apple/cisco-umbrella-2019-2025/processed")
)
if do_download:
    await lake.download(arrow.get("2018-01-01"), arrow.get("2025-12-31"))
    lake.to_parquet(arrow.get("2018-01-01"), arrow.get("2025-12-31"))

2026-02-17 20:55:00,999 [WARNING / measurement.lake.measurement_lake_controller] 2019/08/31: HTTP 404 — likely gap in data
2026-02-17 20:55:01,012 [WARNING / measurement.lake.measurement_lake_controller] 2024/10/23: HTTP 404 — likely gap in data
2026-02-17 20:55:01,086 [WARNING / measurement.lake.measurement_lake_controller] 2019/08/31.csv does not exist
2026-02-17 20:55:01,284 [WARNING / measurement.lake.measurement_lake_controller] 2024/10/23.csv does not exist
2026-02-17 20:55:01,324 [INFO / measurement.lake.measurement_lake_controller] to_parquet done: 0 converted, 2920 already parquet, 2 missing


In [4]:
def extract_domain(lake, domain, start_year=2018, end_year=2025):
    for year in tqdm.tqdm(range(start_year, end_year + 1), desc=domain, leave=False):
        startdate = f"{year}-01-01"
        enddate = f"{year}-12-31"

        view = lake.query(f"""
            SELECT domain, rank, date
            FROM raw
            WHERE date BETWEEN '{startdate}' AND '{enddate}'
        """)

        df = view.query(
            "view",
            f"""
            SELECT date, MIN(rank) AS best_rank, COUNT(*) AS num_subdomains
            FROM view
            WHERE domain LIKE '%.{domain}' OR domain = '{domain}'
            GROUP BY date
            ORDER BY date
        """,
        ).df()

        lake.save(df, domain, year)

In [6]:
START_YEAR = 2018
END_YEAR = 2025

domains = [
    "google.com",
    "zoom.us",
    "teams.microsoft.com",
    "sharepoint.com",
    "office.com",
    "slack.com",
    "linkedin.com",
    "tumblr.com",
    "twitch.tv",
    "netflix.com",
    "reddit.com",
    "steampowered.com",
    "stackoverflow.com",
    "quora.com",
    "w3schools.com",
    "chegg.com",
    "stackexchange.com",
    "openai.com",
    "claude.ai",
    "deepseek.com",
    "blogspot.com",
    "wordpress.com",
    "nessus.org",
    "nflxso.net",
    "ampproject.org",
]

for domain in tqdm.tqdm(domains):
    extract_domain(lake, domain, START_YEAR, END_YEAR)

SyntaxError: incomplete input (334239384.py, line 15)